## Project Template: Phase 1

Below are some concrete steps that you can take while doing your analysis. This guide isn't "one size fit all" so you will probably not do everything listed. But it still serves as a good "pipeline" for how to do data analysis.

If you do engage in a step, you should clearly mention it in the notebook.

---


## Loading Data

1. Load up your data
2. Decide what you want to predict

### Refresher on Data Types

* Scalar (no transformation needed)
    * Numeric
    * Discrete
        * Ordinal
        * Binary
* Text
    * Bag of Words, TF-IDF, Embeddings
* Sets (e.g. tags)
    * Can't do simple bag of words, since tags can be multi word
    * One hot encoding
* Time series
    * Naive approaches
        * Last value
        * Average, Median
        * Max/min
* Numeric Data that isn't directly interpretable (e.g. geospatial data)

In [27]:
import pandas as pd
import re
from datetime import datetime

In [28]:
df = pd.read_csv('data/US_Accidents_Dec21_updated.csv')
df.head(5)

,ID,Severity,Start_Time,End_Time,Start_Lat,Start_Lng,End_Lat,End_Lng,Distance(mi),Description,...,Roundabout,Station,Stop,Traffic_Calming,Traffic_Signal,Turning_Loop,Sunrise_Sunset,Civil_Twilight,Nautical_Twilight,Astronomical_Twilight
0,A-1,3,2016-02-08 00:37:08,2016-02-08 06:37:08,40.108910,-83.092860,40.112060,-83.031870,3.230,Between Sawmill Rd/Exit 20 and OH-315/Olentang...,...,False,False,False,False,False,False,Night,Night,Night,Night
1,A-2,2,2016-02-08 05:56:20,2016-02-08 11:56:20,39.865420,-84.062800,39.865010,-84.048730,0.747,At OH-4/OH-235/Exit 41 - Accident.,...,False,False,False,False,False,False,Night,Night,Night,Night
2,A-3,2,2016-02-08 06:15:39,2016-02-08 12:15:39,39.102660,-84.524680,39.102090,-84.523960,0.055,At I-71/US-50/Exit 1 - Accident.,...,False,False,False,False,False,False,Night,Night,Night,Day
3,A-4,2,2016-02-08 06:51:45,2016-02-08 12:51:45,41.062130,-81.537840,41.062170,-81.535470,0.123,At Dart Ave/Exit 21 - Accident.,...,False,False,False,False,False,False,Night,Night,Day,Day
4,A-5,3,2016-02-08 07:53:43,2016-02-08 13:53:43,39.172393,-84.492792,39.170476,-84.501798,0.500,At Mitchell Ave/Exit 6 - Accident.,...,False,False,False,False,False,False,Day,Day,Day,Day


In [29]:
# make a copy so we retain original
df_clean = df.copy()

## Exploratory Data Analysis (EDA)

1. Decide if feature selection is needed.
    * Do you have hihgly correlated features?
2. Decide if you have non-scalar attributes.
3. What type of supervised learning is this?
    * Binary Classification
    * Multi-class classification?
    * Ordinal classification [Tricky]
        * Do you want to change this into regression or binarize into binary classification?
    * Regression
4. If doing classification
    1. Decide whether you class variable makes sense.
    2. Figure out what your class balance is
5. Histogram the features
    * Good if distribution is highly skewed
6. Vizualize using reduced dimensions
    * PCA, MVD
    * T-SNE

In [30]:
df_clean.head(5)

,ID,Severity,Start_Time,End_Time,Start_Lat,Start_Lng,End_Lat,End_Lng,Distance(mi),Description,...,Roundabout,Station,Stop,Traffic_Calming,Traffic_Signal,Turning_Loop,Sunrise_Sunset,Civil_Twilight,Nautical_Twilight,Astronomical_Twilight
0,A-1,3,2016-02-08 00:37:08,2016-02-08 06:37:08,40.108910,-83.092860,40.112060,-83.031870,3.230,Between Sawmill Rd/Exit 20 and OH-315/Olentang...,...,False,False,False,False,False,False,Night,Night,Night,Night
1,A-2,2,2016-02-08 05:56:20,2016-02-08 11:56:20,39.865420,-84.062800,39.865010,-84.048730,0.747,At OH-4/OH-235/Exit 41 - Accident.,...,False,False,False,False,False,False,Night,Night,Night,Night
2,A-3,2,2016-02-08 06:15:39,2016-02-08 12:15:39,39.102660,-84.524680,39.102090,-84.523960,0.055,At I-71/US-50/Exit 1 - Accident.,...,False,False,False,False,False,False,Night,Night,Night,Day
3,A-4,2,2016-02-08 06:51:45,2016-02-08 12:51:45,41.062130,-81.537840,41.062170,-81.535470,0.123,At Dart Ave/Exit 21 - Accident.,...,False,False,False,False,False,False,Night,Night,Day,Day
4,A-5,3,2016-02-08 07:53:43,2016-02-08 13:53:43,39.172393,-84.492792,39.170476,-84.501798,0.500,At Mitchell Ave/Exit 6 - Accident.,...,False,False,False,False,False,False,Day,Day,Day,Day


## Cleaning
- convert day/night to 0/1
- convert true/false to 0/1

In [31]:
# convert day/night values
day_night = {
    'Day': 0,
    'Night': 1
}

day_night_cols = ['Sunrise_Sunset', 'Civil_Twilight', 'Nautical_Twilight', 'Astronomical_Twilight']

for col in day_night_cols:
    df_clean[col] = df_clean[col].apply(lambda x: day_night[x] if x in day_night else x)

In [32]:
# convert boolean columns
bool_cols = [col for col in df_clean.columns if df_clean.dtypes[col] == bool]

for col in bool_cols:
    df_clean[col] = df_clean[col].apply(lambda x: 0 if x == False else 1)

In [33]:
df_clean.head(5)

,ID,Severity,Start_Time,End_Time,Start_Lat,Start_Lng,End_Lat,End_Lng,Distance(mi),Description,...,Roundabout,Station,Stop,Traffic_Calming,Traffic_Signal,Turning_Loop,Sunrise_Sunset,Civil_Twilight,Nautical_Twilight,Astronomical_Twilight
0,A-1,3,2016-02-08 00:37:08,2016-02-08 06:37:08,40.108910,-83.092860,40.112060,-83.031870,3.230,Between Sawmill Rd/Exit 20 and OH-315/Olentang...,...,0,0,0,0,0,0,1.0,1.0,1.0,1.0
1,A-2,2,2016-02-08 05:56:20,2016-02-08 11:56:20,39.865420,-84.062800,39.865010,-84.048730,0.747,At OH-4/OH-235/Exit 41 - Accident.,...,0,0,0,0,0,0,1.0,1.0,1.0,1.0
2,A-3,2,2016-02-08 06:15:39,2016-02-08 12:15:39,39.102660,-84.524680,39.102090,-84.523960,0.055,At I-71/US-50/Exit 1 - Accident.,...,0,0,0,0,0,0,1.0,1.0,1.0,0.0
3,A-4,2,2016-02-08 06:51:45,2016-02-08 12:51:45,41.062130,-81.537840,41.062170,-81.535470,0.123,At Dart Ave/Exit 21 - Accident.,...,0,0,0,0,0,0,1.0,1.0,0.0,0.0
4,A-5,3,2016-02-08 07:53:43,2016-02-08 13:53:43,39.172393,-84.492792,39.170476,-84.501798,0.500,At Mitchell Ave/Exit 6 - Accident.,...,0,0,0,0,0,0,0.0,0.0,0.0,0.0


## Preprocessing

1. Remove meaningless features (e.g. IDs), or unfair features (e.g. percent grade should be removed if predicing final grade)
2. Discretization
3. Transform features into usable formats (standardize dates, vectorize words)
4. Transform data to a wide format (one row per prediction)
5. **Feature Selection**: Remove redundant, noisy features or unhelpful features
6. Feature creation
    * Use an external tool (e.g. analyzing sentiment from text)
7. Revist EDA using processed features

In [34]:
df_clean.columns

Index(['ID', 'Severity', 'Start_Time', 'End_Time', 'Start_Lat', 'Start_Lng',
       'End_Lat', 'End_Lng', 'Distance(mi)', 'Description', 'Number', 'Street',
       'Side', 'City', 'County', 'State', 'Zipcode', 'Country', 'Timezone',
       'Airport_Code', 'Weather_Timestamp', 'Temperature(F)', 'Wind_Chill(F)',
       'Humidity(%)', 'Pressure(in)', 'Visibility(mi)', 'Wind_Direction',
       'Wind_Speed(mph)', 'Precipitation(in)', 'Weather_Condition', 'Amenity',
       'Bump', 'Crossing', 'Give_Way', 'Junction', 'No_Exit', 'Railway',
       'Roundabout', 'Station', 'Stop', 'Traffic_Calming', 'Traffic_Signal',
       'Turning_Loop', 'Sunrise_Sunset', 'Civil_Twilight', 'Nautical_Twilight',
       'Astronomical_Twilight'],
      dtype='object')

In [35]:
# use regular expressions to make sure all dates are formatted the same (remove .00000000 from the end)
df_clean['Start_Time'] = df_clean['Start_Time'].apply(lambda x: re.sub('([0-9]{4}-[0-9]{2}-[0-9]{2} [0-9]{2}:[0-9]{2}:[0-9]{2})\.[0-9]+', '\\1', x))
df_clean['End_Time'] = df_clean['End_Time'].apply(lambda x: re.sub('([0-9]{4}-[0-9]{2}-[0-9]{2} [0-9]{2}:[0-9]{2}:[0-9]{2})\.[0-9]+', '\\1', x))

In [36]:
# split dates out into year, month, day, hour, minute, second
df_clean['start_year'] = df_clean['Start_Time'].apply(lambda x: datetime.strptime(x, '%Y-%m-%d %H:%M:%S').year)
df_clean['start_month'] = df_clean['Start_Time'].apply(lambda x: datetime.strptime(x, '%Y-%m-%d %H:%M:%S').month)
df_clean['start_day'] = df_clean['Start_Time'].apply(lambda x: datetime.strptime(x, '%Y-%m-%d %H:%M:%S').day)
df_clean['start_hour'] = df_clean['Start_Time'].apply(lambda x: datetime.strptime(x, '%Y-%m-%d %H:%M:%S').hour)
df_clean['start_minute'] = df_clean['Start_Time'].apply(lambda x: datetime.strptime(x, '%Y-%m-%d %H:%M:%S').minute)
df_clean['start_second'] = df_clean['Start_Time'].apply(lambda x: datetime.strptime(x, '%Y-%m-%d %H:%M:%S').second)

df_clean['end_year'] = df_clean['End_Time'].apply(lambda x: datetime.strptime(x, '%Y-%m-%d %H:%M:%S').year)
df_clean['end_month'] = df_clean['End_Time'].apply(lambda x: datetime.strptime(x, '%Y-%m-%d %H:%M:%S').month)
df_clean['end_day'] = df_clean['End_Time'].apply(lambda x: datetime.strptime(x, '%Y-%m-%d %H:%M:%S').day)
df_clean['end_hour'] = df_clean['End_Time'].apply(lambda x: datetime.strptime(x, '%Y-%m-%d %H:%M:%S').hour)
df_clean['end_minute'] = df_clean['End_Time'].apply(lambda x: datetime.strptime(x, '%Y-%m-%d %H:%M:%S').minute)
df_clean['end_second'] = df_clean['End_Time'].apply(lambda x: datetime.strptime(x, '%Y-%m-%d %H:%M:%S').second)

In [37]:
# drop unused columns
df_clean = df_clean.drop(columns=[
    'ID', 
    'Start_Time', 
    'End_Time', 
    'Description',
    'Number', 
    'Street', 
    'Side', 
    'City', 
    'County', 
    'State', 
    'Zipcode', 
    'Country', 
    'Timezone',
    'Airport_Code',
    'Weather_Condition',
    'Amenity',
    'Bump',
    'Crossing',
    'Give_Way',
    'Junction',
    'No_Exit',
    'Railway',
    'Roundabout',
    'Station',
    'Stop',
    'Traffic_Calming',
    'Traffic_Signal',
    'Turning_Loop',
    'Start_Lat',
    'Start_Lng',
    'End_Lat',
    'End_Lng',
    'Wind_Direction',
    'Weather_Timestamp'
])

In [38]:
df_clean.fillna(method='ffill', inplace=True)
df_clean

,Severity,Distance(mi),Temperature(F),Wind_Chill(F),Humidity(%),Pressure(in),Visibility(mi),Wind_Speed(mph),Precipitation(in),Sunrise_Sunset,...,start_day,start_hour,start_minute,start_second,end_year,end_month,end_day,end_hour,end_minute,end_second
0,3,3.230,42.1,36.1,58.0,29.76,10.0,10.4,0.00,1.0,...,8,0,37,8,2016,2,8,6,37,8
1,2,0.747,36.9,36.1,91.0,29.68,10.0,10.4,0.02,1.0,...,8,5,56,20,2016,2,8,11,56,20
2,2,0.055,36.0,36.1,97.0,29.70,10.0,10.4,0.02,1.0,...,8,6,15,39,2016,2,8,12,15,39
3,2,0.123,39.0,36.1,55.0,29.65,10.0,10.4,0.02,1.0,...,8,6,51,45,2016,2,8,12,51,45
4,3,0.500,37.0,29.8,93.0,29.69,10.0,10.4,0.01,0.0,...,8,7,53,43,2016,2,8,13,53,43
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2845337,2,0.543,86.0,86.0,40.0,28.92,10.0,13.0,0.00,0.0,...,23,18,3,25,2019,8,23,18,32,1
2845338,2,0.338,70.0,70.0,73.0,29.39,10.0,6.0,0.00,0.0,...,23,19,11,30,2019,8,23,19,38,23
2845339,2,0.561,73.0,73.0,64.0,29.74,10.0,10.0,0.00,0.0,...,23,19,0,21,2019,8,23,19,28,49
2845340,2,0.772,71.0,71.0,81.0,29.62,10.0,8.0,0.00,0.0,...,23,19,0,21,2019,8,23,19,29,42


In [39]:
# show counts by year
df_clean['start_year'].value_counts()

2021    1511745
2020     625864
2019     258615
2017     163918
2018     163176
2016     122024
Name: start_year, dtype: int64

In [40]:
# finally, drop year columns
df_clean = df_clean.drop(columns=['start_year', 'end_year'])

In [41]:
df_clean.to_csv('data/clean.csv', index=False)

# GOAL: predict severity based on weather conditions and time of day